# Fake Reviews Detection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Fake_reviews

/content/drive/MyDrive/Fake_reviews


In [ ]:
ls

fake_reviews.csv              manual_testing.csv
Fake_reviews_detection.ipynb  true_reviews.csv


### Importing required libraries
Here I am going to importing some of the required library, if extra library is required to install It will be install later on.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

### Inserting fake and real dataset

In [ ]:
df_fake = pd.read_csv("fake_reviews.csv")
df_true = pd.read_csv("true_reviews.csv")

In [ ]:
df_fake.head(5)

,reviewID,reviewerID,restaurantID,date,rating,reviewUsefulCount,reviewContent,flagged,name,location,...,coolCount,funnyCount,complimentCount,tipCount,fanCount,restaurantRating,mnr,rl,rd,Maximum Content Similarity
0,6iHxiTdunSlX1Fg_nGIEeg,MXfwbHv5T-U_lHix5p8cmQ,pbEiXam9YJL3neCYHGwLUA,7/24/2012,5,0,i finally made alinea i heard much alinea way ...,Y,Betty The Star R.,"Chicago, IL",...,3,3,0,0,0,4.5,0.083333,129,0.125,0.134237
1,ZrDMIIfvGb35_MQyZvcNlA,lszTWuvB2yQWCUjcRX73Xw,pbEiXam9YJL3neCYHGwLUA,07-12-2012,5,0,went friend s birthday all i say wow fine dini...,Y,Cindy S.,"Chicago, IL",...,1,0,0,0,0,4.5,0.166667,20,0.125,0.026265
2,vjSYh7i54mbhNPzjKW-75A,jxzkUDFAreOk3VpYFedxZA,WBU0yq9J8qiYQfI_fh2P1Q,8/24/2011,3,0,didn t chicken like folks come here instead i ...,Y,Rich G.,"Chicago, IL",...,3,1,1,0,0,4.0,0.083333,45,0.250,0.041505
3,XKjj-eKn6ZMkGcQ6E3OSXQ,PlKxCQX4aBNx2lp0EyHl0w,8d_DiHWB-pjBVW3a7D8EWg,2/24/2011,5,0,lives hype highly recommended,Y,Frank F.,"Chicago, IL",...,0,0,0,0,0,4.0,0.166667,4,0.250,0.000000
4,7EAzA1FCwmuapu_fI8PyiA,0TqpLe6svu1YUb6zKl2vHg,IvdOI86_CSCRFpid2atnyA,04-06-2011,3,0,it bad server nice little slow although bus pe...,Y,May H.,"Chicago, IL",...,0,0,0,0,0,4.0,0.083333,53,0.250,0.057218


In [ ]:
df_true.head(5)

,reviewID,reviewerID,restaurantID,date,rating,reviewUsefulCount,reviewContent,flagged,name,location,...,coolCount,funnyCount,complimentCount,tipCount,fanCount,restaurantRating,mnr,rl,rd,Maximum Content Similarity
0,GtwU21YOQn-wf4vWRUIx6w,bNYesZ944s6IJVowOnB0iA,pbEiXam9YJL3neCYHGwLUA,9/22/2012,5,0,unlike next we d eaten previous night dish com...,N,Scott E.,"Glengary, WV",...,5,5,2,0,1,4.5,0.083333,497,0.125,0.123653
1,0LpVTc3,TRKxLC3y-ZvP45e5iilMtw,pbEiXam9YJL3neCYHGwLUA,9/22/2012,5,0,probably one best meals i ve ever it s perform...,N,Jerry K.,"Palos Verdes Peninsula, CA",...,0,0,0,0,0,4.5,0.083333,41,0.125,0.000000
2,tljtLzf68Fkwf,0EMm8umAqXZzyhxNpL4M9g,pbEiXam9YJL3neCYHGwLUA,9/19/2012,3,2,service impeccable experience presentation coo...,N,Patricia M.,"Chicago, IL",...,0,1,0,0,0,4.5,0.083333,27,0.375,0.000000
3,iSN,DlwexC7z88ymAzu45skODw,pbEiXam9YJL3neCYHGwLUA,09-06-2012,3,8,the problem places like this given exhorbitant...,N,Terry N.,"San Jose, CA",...,8,6,6,11,0,4.5,0.083333,244,0.375,0.000000
4,Jmwrh7,kW2dk1CWihmh3g7k9N2G8A,pbEiXam9YJL3neCYHGwLUA,09-09-2012,5,1,i idea write review dining alinea brings whole...,N,Shradha Vegetarianista A.,"Chicago, IL",...,679,417,283,22,70,4.5,0.083333,97,0.125,0.760866


Inserting a column called "class" for fake and real news dataset to categories fake and true news. 

In [ ]:
df_fake["class"] = 0
df_true["class"] = 1

In [ ]:
df_fake.shape, df_true.shape

((6206, 26), (20752, 26))

Merging the main fake and true dataframe

In [ ]:
df_marge = pd.concat([df_fake, df_true], axis =0 )
df_marge.head(10)

,reviewID,reviewerID,restaurantID,date,rating,reviewUsefulCount,reviewContent,flagged,name,location,...,funnyCount,complimentCount,tipCount,fanCount,restaurantRating,mnr,rl,rd,Maximum Content Similarity,class
0,6iHxiTdunSlX1Fg_nGIEeg,MXfwbHv5T-U_lHix5p8cmQ,pbEiXam9YJL3neCYHGwLUA,7/24/2012,5,0,i finally made alinea i heard much alinea way ...,Y,Betty The Star R.,"Chicago, IL",...,3,0,0,0,4.5,0.083333,129,0.125,0.134237,0
1,ZrDMIIfvGb35_MQyZvcNlA,lszTWuvB2yQWCUjcRX73Xw,pbEiXam9YJL3neCYHGwLUA,07-12-2012,5,0,went friend s birthday all i say wow fine dini...,Y,Cindy S.,"Chicago, IL",...,0,0,0,0,4.5,0.166667,20,0.125,0.026265,0
2,vjSYh7i54mbhNPzjKW-75A,jxzkUDFAreOk3VpYFedxZA,WBU0yq9J8qiYQfI_fh2P1Q,8/24/2011,3,0,didn t chicken like folks come here instead i ...,Y,Rich G.,"Chicago, IL",...,1,1,0,0,4.0,0.083333,45,0.250,0.041505,0
3,XKjj-eKn6ZMkGcQ6E3OSXQ,PlKxCQX4aBNx2lp0EyHl0w,8d_DiHWB-pjBVW3a7D8EWg,2/24/2011,5,0,lives hype highly recommended,Y,Frank F.,"Chicago, IL",...,0,0,0,0,4.0,0.166667,4,0.250,0.000000,0
4,7EAzA1FCwmuapu_fI8PyiA,0TqpLe6svu1YUb6zKl2vHg,IvdOI86_CSCRFpid2atnyA,04-06-2011,3,0,it bad server nice little slow although bus pe...,Y,May H.,"Chicago, IL",...,0,0,0,0,4.0,0.083333,53,0.250,0.057218,0
5,AzAoPQFysE4wTVWdurM0Gw,mN25MamB30-gtCC-GbvSwA,IvdOI86_CSCRFpid2atnyA,1/22/2011,5,0,was last night booo ya,Y,Matt Matty Boy W.,"Chicago, IL",...,2,0,0,0,4.0,0.083333,5,0.250,0.000000,0
6,H9YMpyJNjZQa1a0J5hgQeQ,bcfG-w1oEhRSRzIcQm_Bxw,IvdOI86_CSCRFpid2atnyA,02-08-2012,4,0,i opportunity try place friends family recent ...,Y,Jimmy L.,"Lombard, IL",...,0,4,0,1,4.0,0.083333,85,0.000,0.047917,0
7,bgZYPS687aLvVN1Bhih6zQ,EbLoh6p0Vj8ZFKq2ZHsO7A,IvdOI86_CSCRFpid2atnyA,06-02-2011,4,0,last night second visit davanti enoteca i yet ...,Y,Lisa H.,"Evanston, IL",...,1,0,0,0,4.0,0.083333,109,0.000,0.167853,0
8,I73kzk9Q9t7v3B1LnM2y2g,qlGgHX-ODtXhqacSjqAl6w,IvdOI86_CSCRFpid2atnyA,8/15/2012,2,0,if looking authentic italian forget place i mo...,Y,Ross E.,"Chicago, IL",...,1,0,0,0,4.0,0.083333,105,0.500,0.000000,0
9,1YXdmOYRcA5-OMBeEUh6gg,oQIAl5_bKkbnT89qD_UTwg,IvdOI86_CSCRFpid2atnyA,08-08-2012,4,0,nice atmosphere i would liked eat outside hot ...,Y,John B.,"Oak Park, IL",...,0,0,0,0,4.0,0.083333,39,0.000,0.000000,0


In [ ]:
df_marge.columns

Index(['reviewID', 'reviewerID', 'restaurantID', 'date', 'rating',
       'reviewUsefulCount', 'reviewContent', 'flagged', 'name', 'location',
       'yelpJoinDate', 'friendCount', 'reviewCount', 'firstCount',
       'usefulCount', 'coolCount', 'funnyCount', 'complimentCount', 'tipCount',
       'fanCount', 'restaurantRating', 'mnr', 'rl', 'rd',
       'Maximum Content Similarity', 'class'],
      dtype='object')

#### columns other than reviewContent and class are not required for detecting the fake reviews, so I am going to drop the columns.

In [ ]:
df = df_marge.drop(df_marge.columns.difference(['reviewContent','class']), axis = 1)

In [ ]:
df = df.dropna(axis=0, how="any", thresh=None, subset=None, inplace=False)

In [ ]:
df.isnull().sum()

reviewContent    0
class            0
dtype: int64

#### Randomly shuffling the dataframe 

In [ ]:
df = df.sample(frac = 1)

In [ ]:
df.head()

,reviewContent,class
11586,the food service slipping i suggested coming w...,1
3272,last week chicago 6 days due trade show every ...,0
12154,this place changed and that s good thing the f...,1
9439,o m g ahhhmazing we party 6 service great ever...,1
2024,sapori really fantastic there lots italian pla...,1


In [ ]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [ ]:
df.columns

Index(['reviewContent', 'class'], dtype='object')

In [ ]:
df.head()

,reviewContent,class
0,the food service slipping i suggested coming w...,1
1,last week chicago 6 days due trade show every ...,0
2,this place changed and that s good thing the f...,1
3,o m g ahhhmazing we party 6 service great ever...,1
4,sapori really fantastic there lots italian pla...,1


#### Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [ ]:
df["reviewContent"] = df["reviewContent"].apply(wordopt)

#### Defining dependent and independent variable as x and y

In [ ]:
x = df["reviewContent"]
y = df["class"]

#### Splitting the dataset into training set and testing set. 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

#### Convert text to vectors

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

### 1. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [ ]:
pred_lr=LR.predict(xv_test)

In [ ]:
LR.score(xv_test, y_test)

0.7885442944056982

In [ ]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.55      0.16      0.25      1472
           1       0.80      0.96      0.88      5267

    accuracy                           0.79      6739
   macro avg       0.68      0.56      0.56      6739
weighted avg       0.75      0.79      0.74      6739



### 2. Decision Tree Classification

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [ ]:
pred_dt = DT.predict(xv_test)

In [ ]:
DT.score(xv_test, y_test)

0.6833358064994807

In [ ]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.29      0.32      0.31      1472
           1       0.81      0.78      0.79      5267

    accuracy                           0.68      6739
   macro avg       0.55      0.55      0.55      6739
weighted avg       0.69      0.68      0.69      6739



### 3. Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [ ]:
pred_gbc = GBC.predict(xv_test)

In [ ]:
GBC.score(xv_test, y_test)

0.7851313251224217

In [ ]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       0.62      0.04      0.08      1472
           1       0.79      0.99      0.88      5267

    accuracy                           0.79      6739
   macro avg       0.70      0.52      0.48      6739
weighted avg       0.75      0.79      0.70      6739



### 4. Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [ ]:
pred_rfc = RFC.predict(xv_test)

In [ ]:
RFC.score(xv_test, y_test)

0.7814215759014691

In [ ]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.49      0.03      0.06      1472
           1       0.79      0.99      0.88      5267

    accuracy                           0.78      6739
   macro avg       0.64      0.51      0.47      6739
weighted avg       0.72      0.78      0.70      6739



# Model Testing With Manual Entry

### Reviews

In [ ]:
def output_lable(n):
    if n == 0:
        return "Fake Review"
    elif n == 1:
        return "Not A Fake Review"
    
def manual_testing(news):
    testing_news = {"reviewContent":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["reviewContent"] = new_def_test["reviewContent"].apply(wordopt) 
    new_x_test = new_def_test["reviewContent"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [ ]:
reviews = str(input())
manual_testing(reviews)

the food is not good


LR Prediction: Not A Fake Review 
DT Prediction: Fake Review 
GBC Prediction: Not A Fake Review 
RFC Prediction: Not A Fake Review
